In [1]:
!pip install "transformers==4.34.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.23.0" "bitsandbytes==0.41.1" "trl==0.4.7" "safetensors>=0.3.1" ipywidgets wandb --upgrade
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=1 pip install flash-attn --no-build-isolation

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# add parent directory to path
import sys
import os

project_dir = os.getcwd()
parent_dir = os.path.dirname(project_dir)
sys.path.insert(0, parent_dir)

In [3]:
# configs
from huggingface_hub import notebook_login
import wandb

model_id = "meta-llama/Llama-2-7b-chat-hf"
notebook_login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sebastiansosa. Use `wandb login --relogin` to force relogin


In [3]:
# get data and convert to Datasets format
import pandas as pd
from datasets import Dataset, DatasetDict
from gaitor_function_calling.data.data_utils import DataAbstractor
from gaitor_function_calling.data.prompting_utils import INSTRUCTION
data_abstractor = DataAbstractor("production_data-without_summary-only_fc.json", f"pre_ft_eval-2_shot")

train_df = pd.DataFrame(data_abstractor.train_data)
test_df = pd.DataFrame(data_abstractor.test_data)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 833
    })
    test: Dataset({
        features: ['text'],
        num_rows: 17
    })
})

In [5]:
# setup metrics and evaluation
import json
from transformers import EvalPrediction
from gaitor_function_calling.evaluation.evaluation_utils import FunctionCallingMetric, compute_perplexity, get_logits_and_labels
from gaitor_function_calling.data.prompting_utils import INSTRUCTION, json_arguments_from_prompt
import numpy as np

fc_metric = FunctionCallingMetric()
def config_compute_metrics(tokenizer):
    def compute_metrics(pred: EvalPrediction):
        # pred.predictions are a batch of logits
        # pred.label_ids are a batch of tokens
        token_ids = np.argmax(pred.predictions, axis=-1)
        predictions = tokenizer.batch_decode(token_ids, skip_special_tokens=False)
        labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=False)
        
        
    
        # Now use your metric class
        fc_result = fc_metric.run(predictions, labels)  # Implement this method in your class
    
        return {
            "fc_combine": fc_result
        }
    return compute_metrics

def custom_evaluation(eval_dataset, model, tokenizer, epoch, save_prompts_path=False):
    print("Starting custom evaluation.")
    model.eval()  # Set the model to evaluation mode
    results = {}

    fc_results = []
    exact_match_results = []
    perplexity_results = []


    if save_prompts_path:
        try:
            with open(save_prompts_path, "r") as f:
                dpo_data = json.load(f)
        except:
            dpo_data = []


    for idx, example in enumerate(eval_dataset):
        print(f"Example {idx}: ", end="")
        post_message = ""
        
        # Custom Function Calling metric
        prompts = None
        try:
            generated_arguments, expected_arguments, prompts = json_arguments_from_prompt(
                example["text"],
                model,
                tokenizer,
                INSTRUCTION,
                {"idx": idx, "epoch": epoch}
            )
            fc_result = fc_metric.run(generated_arguments, expected_arguments)

            fc_results.append(fc_result)

            if save_prompts_path:
                # add prompts to dpo_data.json
                dpo_data.append({
                    "fc_result": fc_result,
                    **prompts
                })
                with open(save_prompts_path, "w") as f:
                    json.dump(dpo_data, f)
        except Exception as e:
            post_message += f"Error function calling: {e}\n"
            fc_results.append(0)

        # exact match metric
        if prompts:
            exact_match_res = fc_metric._sentence_similarity(prompts["expected_str"].split("[/INST]")[-1], prompts["generated_str"].split("[/INST]")[-1])
            exact_match_results.append(exact_match_res)
        else:
            exact_match_results.append(0)

        # perplexity metric
        try:
            logits, labels = get_logits_and_labels(example["text"], model, tokenizer)
            perplexity = compute_perplexity(logits[..., :-1, :], labels).item()
            perplexity_results.append(perplexity)
        except Exception as e:
            post_message += f"Error perplexity: {e}\n"
            # perplexity_results.append(float('inf'))

        example_metric = {
            "fc_combine": fc_results[-1],
            "exact_match": exact_match_results[-1],
            "perplexity": perplexity_results[-1] if len(perplexity_results) else None
        }
        print(example_metric)
        if post_message:
            print(post_message)
        
    results["fc_combine"] =  sum(fc_results) / len(fc_results)
    results["exact_match"] =  sum(exact_match_results) / len(exact_match_results)
    results["perplexity"] =  sum(perplexity_results) / len(perplexity_results) if len(perplexity_results) else 1

    return results

In [6]:
# base LLM model and tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
prompts_path = "./gaitor_function_calling/data/prompts_data-2_shot.json"

eval_result = custom_evaluation(dataset_dict["train"], model, tokenizer, 0, prompts_path)
print(f"Evaluation results train: {eval_result}")
eval_result = custom_evaluation(dataset_dict["test"], model, tokenizer, 0, prompts_path)
print(f"Evaluation results test: {eval_result}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Starting evaluation 0 shot.
Starting custom evaluation.
Example 0: {'fc_combine': 0.3577747344970703, 'exact_match': 0.8500394225120544, 'perplexity': 6.773812294006348}
Example 1: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 6.861993789672852}
Error function calling: 'NoneType' object has no attribute 'group'

Example 2: {'fc_combine': 0.36043891310691833, 'exact_match': 0.645621657371521, 'perplexity': 7.1938652992248535}
Example 3: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.170538902282715}
Error function calling: No function call found in generated data

Example 4: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.170538902282715}
Error function calling: 'NoneType' object has no attribute 'group'

Example 5: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 14.873311996459961}
Error function calling: No function call found in generated data

Example 6: {'fc_combine': 0, 'exact_match': 0.8006242513656616, 'perplexity': 14.873311996459961}
Error function calling: divi

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▃▆█
exact_match,▁▅█▄
fc_combine,▁▅█▅
perplexity,█▄▄▁
epoch,3
exact_match,0.54072
fc_combine,0.27693
perplexity,5.50439
